<a href="https://colab.research.google.com/github/mehmetbozdemir24/Tubitak_1505_Proje/blob/chunking/1505_project_Chunker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import os
import sys
import pickle
import subprocess
import re

# Gerekli paketlerin kontrolü ve kurulumu
def install_packages():
    print("Gerekli kütüphaneler kuruluyor... Lütfen bekleyin.")
    packages = [
        "langchain",
        "langchain-community",
        "pypdf",
        "openpyxl",
        "pandas",
        "python-docx",
        "python-pptx",
        "unstructured",
        "networkx"
    ]
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install"] + packages)
        print("Kurulum tamamlandı.")
    except subprocess.CalledProcessError as e:
        print(f"Kurulum sırasında bir hata oluştu: {e}")
        # Optionally re-raise or handle more gracefully

try:
    import langchain
    import pandas as pd
    from docx import Document as DocxDocument
    from langchain_community.document_loaders import (
        PyPDFLoader,
        UnstructuredPowerPointLoader
    )
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    from langchain_core.documents import Document
except ImportError:
    install_packages()
    # Re-attempt imports after installation
    import langchain
    import pandas as pd
    from docx import Document as DocxDocument
    from langchain_community.document_loaders import (
        PyPDFLoader,
        UnstructuredPowerPointLoader
    )
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    from langchain_core.documents import Document


In [29]:
import os

# Kodun çalıştığı klasör (C:\mebi\Tubitak_1505_Proje)
current_dir = os.getcwd()

# HATA BURADAYDI: project_root = os.path.dirname(current_dir) yapınca C:\mebi'ye çıkıyordu.
# DÜZELTME: Olduğumuz yer zaten proje ana klasörü, yukarı çıkma.
project_root = current_dir

# Doküman klasörünün yolu
BASE_PATH = os.path.join(project_root, "Doküman")
TARGET_FOLDERS = ["pdf", "excel", "word", "powerpoint"]

print(f"Şu anki konum: {current_dir}")
print(f"Hedef klasör (BASE_PATH): {BASE_PATH}")

# Main fonksiyonu içindeki output_file kısmını da buna göre güncellemelisin:
# output_file = os.path.join(project_root, "tum_dokumanlar_final_last.pkl")

Şu anki konum: C:\mebi\Tubitak_1505_Proje
Hedef klasör (BASE_PATH): C:\mebi\Tubitak_1505_Proje\Doküman


In [30]:
def detect_permission_from_content(text_content):

    text_lower = text_content.lower()[:5000]

    # 1. Seviye: Admin
    admin_keywords = ["gizli", "confidential", "passwords", "şifreler", "yönetim kurulu", "admin only"]
    if any(k in text_lower for k in admin_keywords):
        return "admin"

    # 2. Seviye: Manager
    manager_keywords = ["maliyet", "bütçe", "budget", "finans", "salary", "maaş", "forecast"]
    if any(k in text_lower for k in manager_keywords):
        return "manager"

    # 3. Seviye: Editor
    editor_keywords = ["taslak", "draft", "düzenlenecek", "teknik şartname"]
    if any(k in text_lower for k in editor_keywords):
        return "editor"

    # Varsayılan: User
    return "user"



In [31]:
class DriveDocumentProcessor:
    def __init__(self, chunk_size=1000, chunk_overlap=100):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        self.processed_chunks = []

    def process_excel(self, file_path):
        """
        Excel dosyasındaki her sayfayı tek bir Markdown tablosu olarak işler.
        Böylece LLM tablo bütünlüğünü görerek matematiksel işlemler yapabilir.
        """
        try:
            # Excel dosyasını oku
            xls = pd.read_excel(file_path, sheet_name=None)
            filename = os.path.basename(file_path)

            for sheet_name, df in xls.items():
                # 1. Boş satır ve sütunları temizle
                df.dropna(how='all', axis=0, inplace=True)
                df.dropna(how='all', axis=1, inplace=True)

                # 2. Tarih formatlarını düzelt (datetime objelerini string'e çevir)
                for col in df.columns:
                    if pd.api.types.is_datetime64_any_dtype(df[col]):
                        df[col] = df[col].dt.strftime('%Y-%m-%d')

                # 3. VERİYİ MARKDOWN TABLOYA ÇEVİR (En kritik adım burası)
                # index=False diyerek gereksiz satır numaralarını (0,1,2...) almiyoruz.
                try:
                    markdown_table = df.to_markdown(index=False)
                except ImportError:
                    print("HATA: 'tabulate' kütüphanesi eksik. Lütfen 'pip install tabulate' çalıştırın.")
                    return

                # Yetki analizi (tablo içeriğine göre)
                permission = detect_permission_from_content(markdown_table)

                # 4. Tek bir Chunk oluştur (Bütün tablo tek parça)
                table_text = (
                    f"KAYNAK: {filename} > {sheet_name}\n"
                    f"TÜR: Excel Tablosu\n"
                    f"İÇERİK:\n"
                    f"{markdown_table}"
                )

                metadata = {
                    "source": filename,
                    "sheet": sheet_name,
                    "file_type": "excel", # Tipini değiştirdik, artık satır değil tablo.
                    "permission": permission,
                    "columns_present": df.columns.tolist()
                }

                # self.text_splitter kullanmıyoruz çünkü tabloyu bölmek istemiyoruz.
                # Doğrudan listeye ekliyoruz.
                self.processed_chunks.append(Document(page_content=table_text, metadata=metadata))

            print(f"  -> Excel Tablo formatında işlendi: {filename} ({len(xls)} sayfa)")

        except Exception as e:
            print(f"  !! Excel hatası ({os.path.basename(file_path)}): {e}")

    def process_word_detailed(self, file_path):
        """
        Word dosyasını açar, paragrafları ayrı, tabloları ayrı işler.
        Tabloları Excel mantığıyla satır satır anlamlandırır.
        """
        try:
            doc = DocxDocument(file_path)
            filename = os.path.basename(file_path)

            full_text = "\n".join([p.text for p in doc.paragraphs])
            permission = detect_permission_from_content(full_text)

            text_content = []
            for para in doc.paragraphs:
                if para.text.strip():
                    text_content.append(para.text)

            if text_content:
                raw_text = "\n".join(text_content)
                text_chunks = self.text_splitter.create_documents([raw_text])
                for chunk in text_chunks:
                    chunk.metadata.update({
                        "source": filename,
                        "file_type": "word",
                        "permission": permission
                    })
                    self.processed_chunks.append(chunk)

            for i, table in enumerate(doc.tables):

                if not table.rows: continue

                headers = [cell.text.strip() for cell in table.rows[0].cells]

                if all(h == "" for h in headers):
                    headers = [f"Sütun_{k+1}" for k in range(len(headers))]

                for j, row in enumerate(table.rows[1:], start=1):
                    row_content = []
                    for k, cell in enumerate(row.cells):
                        if k < len(headers):
                            key = headers[k]
                            val = cell.text.strip()
                            if val:
                                row_content.append(f"{key}: {val}")

                    if row_content:

                        table_text = (
                            f"BELGE: {filename} > TABLO_{i+1}\n"
                            f"SATIR: {j}\n"
                            f"İÇERİK:\n" + "\n".join(row_content)
                        )

                        meta = {
                            "source": filename,
                            "type": "word_table",
                            "table_id": i+1,
                            "row_id": j,
                            "permission": permission
                        }
                        self.processed_chunks.append(Document(page_content=table_text, metadata=meta))

            print(f"  -> Word detaylı işlendi: {filename}")

        except Exception as e:
            print(f"  !! Word hatası ({filename}): {e}")

    def process_powerpoint_pdf(self, file_path, file_type):
        loader = None
        try:
            if file_type == "pdf":
                loader = PyPDFLoader(file_path)
            elif file_type == "powerpoint":
                loader = UnstructuredPowerPointLoader(file_path)

            if not loader: return

            raw_docs = loader.load()

            # İçerikten yetki analizi
            full_sample_text = " ".join([d.page_content for d in raw_docs[:5]])
            permission = detect_permission_from_content(full_sample_text)

            chunks = self.text_splitter.split_documents(raw_docs)
            filename = os.path.basename(file_path)

            for chunk in chunks:
                page = chunk.metadata.get("page", chunk.metadata.get("page_number", 1))
                chunk.metadata.update({
                    "source": filename,
                    "permission": permission,
                    "file_type": file_type
                })
                self.processed_chunks.append(chunk)

            print(f"  -> {file_type.upper()} işlendi: {len(chunks)} parça. (Yetki: {permission})")

        except Exception as e:
            print(f"  !! Hata ({file_type} - {os.path.basename(file_path)}): {e}")

    def save_data(self, output_path):
        with open(output_path, "wb") as f:
            pickle.dump(self.processed_chunks, f)
        print(f"\nToplam {len(self.processed_chunks)} chunk kaydedildi: {output_path}")


In [32]:
def main():
    processor = DriveDocumentProcessor()
    print(f"Tarama Başlıyor: {BASE_PATH}")

    if not os.path.exists(BASE_PATH):
        print(f"HATA: Belirtilen yol bulunamadı! -> {BASE_PATH}")

        return

    for folder in TARGET_FOLDERS:
        folder_path = os.path.join(BASE_PATH, folder)
        if not os.path.exists(folder_path):
            print(f"Uyarı: Klasör yok -> {folder}")
            continue

        print(f"\n[{folder.upper()}] Klasörü Taranıyor...")
        files = os.listdir(folder_path)

        for filename in files:
            file_path = os.path.join(folder_path, filename)

            # Gizli dosyaları atla
            if filename.startswith("."): continue

            # Dosya türüne göre işlem
            if folder == "excel" and (filename.endswith(".xlsx") or filename.endswith(".xls")):
                processor.process_excel(file_path)

            elif folder == "word" and (filename.endswith(".docx") or filename.endswith(".doc")):
                processor.process_word_detailed(file_path)

            elif folder == "pdf" and filename.endswith(".pdf"):
                processor.process_powerpoint_pdf(file_path, "pdf")

            elif folder == "powerpoint" and (filename.endswith(".pptx") or filename.endswith(".ppt")):
                processor.process_powerpoint_pdf(file_path, "powerpoint")

    output_file = os.path.join(project_root, "tum_dokumanlar_final_last.pkl")

    processor.save_data(output_file)

if __name__ == "__main__":
    main()

Tarama Başlıyor: C:\mebi\Tubitak_1505_Proje\Doküman

[PDF] Klasörü Taranıyor...
  -> PDF işlendi: 5 parça. (Yetki: admin)
  -> PDF işlendi: 22 parça. (Yetki: user)
  -> PDF işlendi: 27 parça. (Yetki: admin)
  -> PDF işlendi: 22 parça. (Yetki: admin)
  -> PDF işlendi: 44 parça. (Yetki: user)
  -> PDF işlendi: 51 parça. (Yetki: user)
  -> PDF işlendi: 13 parça. (Yetki: user)
  -> PDF işlendi: 111 parça. (Yetki: user)
  -> PDF işlendi: 24 parça. (Yetki: user)
  -> PDF işlendi: 53 parça. (Yetki: manager)
  -> PDF işlendi: 67 parça. (Yetki: user)
  -> PDF işlendi: 36 parça. (Yetki: user)
  -> PDF işlendi: 8 parça. (Yetki: admin)
  -> PDF işlendi: 14 parça. (Yetki: user)
  -> PDF işlendi: 5 parça. (Yetki: user)
  -> PDF işlendi: 7 parça. (Yetki: admin)
  -> PDF işlendi: 3 parça. (Yetki: user)
  -> PDF işlendi: 7 parça. (Yetki: user)
  -> PDF işlendi: 5 parça. (Yetki: user)
  -> PDF işlendi: 15 parça. (Yetki: user)
  -> PDF işlendi: 3 parça. (Yetki: user)
  -> PDF işlendi: 3 parça. (Yetki: us

In [27]:
import torch
print("Yüklü Sürüm:", torch.__version__)
# Çıktıda '+cpu' YERİNE '+cu121' yazmalı.

print("CUDA Var mı?:", torch.cuda.is_available())
# Burası artık True olmalı.

Yüklü Sürüm: 2.8.0+cpu
CUDA Var mı?: False
